In [1]:
import os
import time
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("CUDA available:", torch.cuda.is_available())
print("Device:", device)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


CUDA available: True
Device: cuda
GPU: NVIDIA RTX 6000 Ada Generation


In [3]:
def smart_read_csv(path):
    for sep in [";", "\t", ","]:
        try:
            df = pd.read_csv(path, sep=sep)
            if df.shape[1] > 1:
                return df
        except:
            pass
    raise ValueError(f"Impossible de lire {path}")
def find_time_col(df):
    for c in df.columns:
        if "time" in c.lower():
            return c
    raise ValueError("Colonne temps non trouvée")
def read_and_slice_by_time(path, t0, t1):
    df = smart_read_csv(path)
    tcol = find_time_col(df)

    df = df[(df[tcol] >= t0) & (df[tcol] <= t1)]
    df = df.drop(columns=[tcol])

    return torch.tensor(df.values, dtype=torch.float32)


In [4]:
def resample_to_L(x, L):
    T, C = x.shape

    if T == 0:
        return torch.zeros(L, C)

    if T == L:
        return x

    idx = torch.linspace(0, T - 1, L)
    idx0 = idx.long()
    idx1 = torch.clamp(idx0 + 1, max=T - 1)
    w = idx - idx0

    return (1 - w).unsqueeze(1) * x[idx0] + w.unsqueeze(1) * x[idx1]


In [5]:
ROOTS = {
    "plantar": "/home/fisa/stockage1/mindscan/Plantar_activity",
    "events":  "/home/fisa/stockage1/mindscan/Events"
}

FILENAMES = {
    "plantar": "insoles.csv"
}

L = 256


In [6]:
def build_segments_index(events_root):
    rows = []

    subjects = sorted([
        d for d in os.listdir(events_root)
        if os.path.isdir(os.path.join(events_root, d))
    ])

    for subject in subjects:
        for seq in os.listdir(os.path.join(events_root, subject)):
            classif = os.path.join(events_root, subject, seq, "classif.csv")
            if not os.path.exists(classif):
                continue

            df = pd.read_csv(classif, sep=";")
            for _, r in df.iterrows():
                rows.append({
                    "subject": subject,
                    "seq": seq,
                    "label": int(r["Class"]),
                    "t0": float(r["Timestamp Start"]),
                    "t1": float(r["Timestamp End"]),
                })

    return pd.DataFrame(rows)
segments = build_segments_index(ROOTS["events"])
print("Total segments:", len(segments))


Total segments: 10204


In [7]:
allowed_subjects = {f"S{str(i).zfill(2)}" for i in range(1, 25)}
segments = segments[segments["subject"].isin(allowed_subjects)].reset_index(drop=True)

subjects = sorted(segments["subject"].unique())
np.random.seed(0)
np.random.shuffle(subjects)

n_train = int(0.8 * len(subjects))
train_subjects = set(subjects[:n_train])
val_subjects   = set(subjects[n_train:])

train_segments = segments[segments["subject"].isin(train_subjects)].reset_index(drop=True)
val_segments   = segments[segments["subject"].isin(val_subjects)].reset_index(drop=True)

print("Train segments:", len(train_segments))
print("Val segments:", len(val_segments))


Train segments: 6084
Val segments: 1569


In [8]:
class PlantarEventDataset(Dataset):
    def __init__(self, segments_df, roots, filenames, L=256):
        self.df = segments_df.reset_index(drop=True)
        self.roots = roots
        self.filenames = filenames
        self.L = L

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        r = self.df.iloc[i]
        subject, seq = r["subject"], r["seq"]
        t0, t1 = float(r["t0"]), float(r["t1"])
        y = int(r["label"]) - 1

        path = os.path.join(
            self.roots["plantar"], subject, seq, self.filenames["plantar"]
        )

        x = read_and_slice_by_time(path, t0, t1)   # [T, C]
        x = resample_to_L(x, self.L)               # [256, C]
        x = x.transpose(0, 1).contiguous()         # [C, 256]

        return {
            "plantar": x,
            "y": torch.tensor(y, dtype=torch.long)
        }


In [9]:
train_ds = PlantarEventDataset(train_segments, ROOTS, FILENAMES, L)
val_ds   = PlantarEventDataset(val_segments, ROOTS, FILENAMES, L)

train_dl = DataLoader(
    train_ds,
    batch_size=128,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True
)

val_dl = DataLoader(
    val_ds,
    batch_size=128,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True
)
b = next(iter(train_dl))
print("Train batch:", b["plantar"].shape, b["y"].shape)

b = next(iter(val_dl))
print("Val batch:", b["plantar"].shape, b["y"].shape)


Train batch: torch.Size([128, 50, 256]) torch.Size([128])
Val batch: torch.Size([128, 50, 256]) torch.Size([128])


In [10]:
class Plantar_CNN(nn.Module):
    def __init__(self, in_channels, num_classes=31):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv1d(in_channels, 64, 7, stride=2, padding=3),
            nn.ReLU(),
            nn.Conv1d(64, 128, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv1d(128, 256, 3, stride=2, padding=1),
            nn.ReLU(),
        )

        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.mean(dim=-1)
        return self.classifier(x)


In [25]:
def train_plantar(model, train_dl, val_dl, epochs=5):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        correct = total = 0
        running_loss = 0.0

        pbar = tqdm(train_dl, desc=f"Epoch {epoch+1}/{epochs}")
        for batch in pbar:
            x = batch["plantar"].to(device, non_blocking=True)
            y = batch["y"].to(device, non_blocking=True)

            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.size(0)
            correct += (logits.argmax(1) == y).sum().item()
            total += x.size(0)

            pbar.set_postfix(
                loss=f"{running_loss/total:.4f}",
                acc=f"{correct/total:.3f}"
            )

        # Validation
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for batch in val_dl:
                x = batch["plantar"].to(device)
                y = batch["y"].to(device)
                correct += (model(x).argmax(1) == y).sum().item()
                total += x.size(0)

        print(f"Val acc: {correct/total:.3f}")
        
in_channels = b["plantar"].shape[1]

model = Plantar_CNN(in_channels).to(device)
train_plantar(model, train_dl, val_dl, epochs=10)


Epoch 1/10:   0%|          | 0/48 [00:00<?, ?it/s]

Epoch 1/10: 100%|██████████| 48/48 [00:43<00:00,  1.11it/s, acc=0.263, loss=3.0951]


Val acc: 0.449


Epoch 2/10: 100%|██████████| 48/48 [00:40<00:00,  1.18it/s, acc=0.518, loss=1.6269]


Val acc: 0.562


...

Val acc: 0.664


Epoch 9/10: 100%|██████████| 48/48 [00:42<00:00,  1.13it/s, acc=0.723, loss=0.8751]


Val acc: 0.669


Epoch 10/10: 100%|██████████| 48/48 [00:39<00:00,  1.22it/s, acc=0.743, loss=0.7978]


Val acc: 0.704


In [12]:
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            x = batch["plantar"].to(device)
            y = batch["y"].to(device)

            logits = model(x)
            preds = logits.argmax(dim=1)

            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total
val_acc = evaluate(model, val_dl)
print(f"Validation accuracy (Plantar): {val_acc:.3f}")


Validation accuracy (Plantar): 0.666


In [24]:
sample = val_ds[100]

x = sample["plantar"].unsqueeze(0).to(device)  # [1, C, 256]
y_true = sample["y"].item()

model.eval()
with torch.no_grad():
    logits = model(x)
    y_pred = logits.argmax(dim=1).item()

print("Vrai label :", y_true)
print("Label prédit :", y_pred)



Vrai label : 4
Label prédit : 29
